# Imports

In [6]:
import os
import shutil
import zipfile

from PAL import *
engine = Redshift_Connection('ssopariwalla')

# Pull Leak/UI_LEAK Alerts from March Onwards

In [7]:
FACs_to_exclude = ('Y87BQV9', #AperiaTech
                   'X77OKO7', # Amazon Last Mile
                   '1L10O39N', # Gemini DVT Deployments
                   '1GOPAPMF', # Not Configured
                   '3FF99DS',  # UNKNOWN_FLEET
                   '1FBR7DGI', # TTN
                   '1M5190AC',  # YTX
                   '19RVO6ZP' #Amazon EOL Vehicles	
                   )

In [8]:
schema = account_schema_mapping['AMZ']
query = f"""
    SELECT DISTINCT
        fleet_meta_data.fleet_name,
        vehicle_meta_data.fleet_vehicle_id,
        meta_data.side || meta_data.axle || meta_data.position as tire,
        meta_data.set_point,
        event_table.unique_id,
        event_table.event_id as alert_id,
        event_table.event_type,
        event_table.pressure_date,
        event_table.ts_created as alert_opened_ts,
        sensor_data.pressure as pressure_at_detection
    FROM
        {schema}.fleet_meta_data INNER JOIN {schema}.vehicle_meta_data
            ON fleet_meta_data.fleet_id = vehicle_meta_data.fleet_id
        INNER JOIN {schema}.meta_data
            ON meta_data.vehicle_id = vehicle_meta_data.vehicle_id
        INNER JOIN {schema}.event_table
            ON event_table.unique_id = meta_data.unique_id
        INNER JOIN {schema}.sensor_data
            ON (sensor_data.pressure_date = event_table.pressure_date
                AND sensor_data.unique_id = event_table.unique_id)
    WHERE
        event_table.ts_created >= '2025-03-01'
        AND event_table.event_type IN ('LEAK','UI_LEAK')
        and meta_data.id IN (SELECT MAX(id) FROM {schema}.meta_data GROUP BY unique_id)
        AND fleet_meta_data.code NOT IN {FACs_to_exclude}
    ORDER BY
        event_table.ts_created DESC
"""
# leak_alerts = pd.read_sql(query,engine)
# leak_alerts.to_pickle('leak_alerts.pkl')
leak_alerts = pd.read_pickle('leak_alerts.pkl')

In [9]:
leaks_caught_early = leak_alerts[leak_alerts['pressure_at_detection']>85].copy()

In [ ]:
output_dir = "alert_plots"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

for index_row, row in leaks_caught_early.iterrows():
    # Query the data
    query = f"""
    SELECT
        pressure_date,
        pressure,
        speed,
        {row['set_point']} as set_point
    FROM
        {schema}.sensor_data
    WHERE
        unique_id = '{row['unique_id']}'
        AND pressure_date >= '{row['pressure_date'] - pd.Timedelta(hours=8)}'
        AND pressure_date <= '{row['pressure_date'] + pd.Timedelta(hours=24*3)}'
    ORDER BY
        pressure_date ASC
    """
    sensor_data = pd.read_sql(query, engine)

    # Create the plot
    fig = go.Figure()

    # Pressure in purple
    fig.add_trace(go.Scatter(
        x=sensor_data['pressure_date'],
        y=sensor_data['pressure'],
        mode='lines',
        name='Pressure',
        line=dict(color='purple')
    ))

    # Speed in grey
    fig.add_trace(go.Scatter(
        x=sensor_data['pressure_date'],
        y=sensor_data['speed'],
        mode='lines',
        name='Speed',
        line=dict(color='grey')
    ))

    # Set point as black dashed
    fig.add_trace(go.Scatter(
        x=sensor_data['pressure_date'],
        y=sensor_data['set_point'],
        mode='lines',
        name='Set Point',
        line=dict(color='black', dash='dash')
    ))
    # Vertical dashed red line at pressure_date
    fig.add_trace(go.Scatter(
        x=[row['pressure_date'], row['pressure_date']],
        y=[0, row['set_point']+5],
        mode='lines',
        name='Pressure Date',
        line=dict(color='red', dash='dash'),
        showlegend=False
    ))
    
    # Vertical solid red line at alert_opened_ts
    fig.add_trace(go.Scatter(
        x=[row['alert_opened_ts'], row['alert_opened_ts']],
        y=[0, row['set_point']+5],
        mode='lines',
        name='Alert Opened',
        line=dict(color='red', dash='solid'),
        showlegend=False
    ))
    # Layout
    fig.update_layout(
        title=f"Alert ID: {row['alert_id']} -- Asset ID: {row['fleet_vehicle_id']} {row['tire']}",
        xaxis_title='Timestamp',
        yaxis_title='Values',
        template='simple_white'
    )
    output_file = os.path.join(output_dir, f"alert_{row['alert_id']}.html")
    fig.write_html(output_file)



In [ ]:
zip_filename = f"{output_dir}.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(output_dir):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, start=output_dir)
            zipf.write(file_path, arcname=arcname)


# Grab Information Only For Verified Leaks

In [10]:
alert_ids = [
    3004532, 3005008, 3005509, 3005839, 3006017, 3006339, 3006545, 3006695, 3006862, 3006912,
    3007586, 3007615, 3007697, 3007835, 3007960, 3008030, 3008133, 3009167, 3009291, 3009339,
    3009617, 3009763, 3009816, 3010227, 3010895, 3011075, 3011099, 3011611, 3011681, 3011942,
    3012427, 3012804, 3014141, 3015152, 3015394, 3015642, 3015688, 3015882, 3016609, 3016631,
    3016963, 3017183, 3017360, 3017767, 3018402, 3019113, 3019183, 3020334, 3020877, 3021086,
    3021461, 3021573, 3021638, 3022135, 3022372, 3022818, 3023130, 3023334
]


In [11]:
leaks_to_share_w_ankit = leaks_caught_early[leaks_caught_early['alert_id'].isin(alert_ids)]

In [12]:
leaks_to_share_w_ankit.sort_values(by=['fleet_vehicle_id','tire','pressure_date'],ascending=True,inplace=True,ignore_index=True)

In [13]:
leaks_to_share_w_ankit = leaks_to_share_w_ankit.drop([14, 21])


In [19]:
output_dir = "Leaks Caught Early"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

for index_row, row in leaks_to_share_w_ankit.iterrows():
    # Query the data
    start = row['pressure_date'] - pd.Timedelta(hours=8)
    end   = row['pressure_date'] + pd.Timedelta(hours=24*3)
    query = f"""
    SELECT
        pressure_date,
        pressure,
        speed,
        {row['set_point']} AS set_point
    FROM
        {schema}.sensor_data
    WHERE
        unique_id = '{row['unique_id']}'
        AND pressure_date BETWEEN '{start}' AND '{end}'
    ORDER BY
        pressure_date ASC
    """
    sensor_data = pd.read_sql(query, engine)
    max_y = max(sensor_data['pressure'].max(), sensor_data['set_point'].max())
    
    # Create the plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=sensor_data['pressure_date'],
        y=sensor_data['pressure'],
        mode='lines',
        name='Pressure',
        line=dict(color='purple')
    ))
    fig.add_trace(go.Scatter(
        x=sensor_data['pressure_date'],
        y=sensor_data['speed'],
        mode='lines',
        name='Speed',
        line=dict(color='grey')
    ))
    fig.add_trace(go.Scatter(
        x=sensor_data['pressure_date'],
        y=sensor_data['set_point'],
        mode='lines',
        name='Set Point',
        line=dict(color='black', dash='dash')
    ))
    # Vertical dashed red line at pressure_date
    fig.add_trace(go.Scatter(
        x=[row['pressure_date'], row['pressure_date']],
        y=[0, max_y + 5],
        mode='lines',
        line=dict(color='red', dash='dash'),
        showlegend=False
    ))

    # Layout
    fig.update_layout(
        title=f"{row['fleet_vehicle_id']} {row['tire']} -- Leak Detected @ {row['pressure_at_detection']} PSI",
        xaxis_title='Timestamp',
        yaxis_title='Pressure',
        template='simple_white',
        showlegend=False  # turn off the legend entirely
    )

    # Write out high-quality JPEG
    jpg_file = os.path.join(output_dir, f"alert_{row['alert_id']}.jpg")
    fig.write_image(
        jpg_file,
        format='jpg',
        width=1280,         # e.g. Full HD width
        height=720,        # e.g. Full HD height
        scale=2             # 2× super-sampling for extra sharpness
    )


In [21]:
zip_filename = f"{output_dir}.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(output_dir):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, start=output_dir)
            zipf.write(file_path, arcname=arcname)


In [23]:
leaks_to_share_w_ankit.rename({
    'fleet_name':'Domicile',
    'fleet_vehicle_id':'Asset ID',
    'tire':'Tire',
    'set_point':'Set Point',
    'alert_id':'Alert ID',
    'event_type': 'Event Type',
    'pressure_date': 'Timestamp Issue Detected',
    'alert_opened_ts': 'Timestamp Alert Sent',
    'pressure_at_detection':'Pressure at Detection'
},axis=1,inplace=True)

In [25]:
leaks_to_share_w_ankit.drop('unique_id',axis=1,inplace=True)

In [27]:
leaks_to_share_w_ankit.to_excel('Leaks Caught Early.xlsx',index = False)